In [ ]:
# Instalando o Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
# instalando a findspark
!pip install -q findspark

In [ ]:
# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()

In [ ]:
# importando o pacote necessário para iniciar uma seção Spark
from pyspark.sql import SparkSession

# iniciando o spark context
sc = SparkSession.builder.master('local[*]').getOrCreate()

# Verificando se a sessão foi criada
sc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# carregando um conjunto de dados que baixamos da internet
df = sc.read.csv(
    path = "/content/MICRODADOS_ENEM_2020.csv", 
    inferSchema = True, 
    header = True,
    sep = ';', 
    encoding = "UTF-8")

In [ ]:
# Verificando o tipo de objeto criado
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Espiando o dataset
df.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

In [ ]:
# Verificando o schema() deste sparkdataframe
df.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [ ]:
#Uma expressão que retorna true se a coluna for NaN.

from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+---------+--------------+--------------+--------------+--------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|

In [ ]:
#média da prova de Linguagens e Códigos por Tipo de escola do Ensino Médio
df.groupBy("TP_ESCOLA").avg("NU_NOTA_LC").show(truncate=False)

+---------+-----------------+
|TP_ESCOLA|avg(NU_NOTA_LC)  |
+---------+-----------------+
|1        |531.0384913566235|
|3        |582.3449100938481|
|2        |522.357320055366 |
+---------+-----------------+



In [ ]:
#média da prova de Matemática por Tipo de escola do Ensino Médio
df.groupBy("TP_ESCOLA").avg("NU_NOTA_MT").show(truncate=False)

+---------+------------------+
|TP_ESCOLA|avg(NU_NOTA_MT)   |
+---------+------------------+
|1        |527.8932670294267 |
|3        |627.7723487881202 |
|2        |507.42500380887003|
+---------+------------------+



In [ ]:
#média da prova de Matemática em geral
from pyspark.sql.functions import mean 
df.select(mean("NU_NOTA_MT")).show(truncate=True)	

+----------------+
| avg(NU_NOTA_MT)|
+----------------+
|528.457666120597|
+----------------+



In [ ]:
df.summary().show()

+-------+--------------------+------+------------------+-------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+----------------+------------------+---------+----------------------+-------------------+-------------------+------------------+--------------------+-----------------+-----------+-------------------+------------------+------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------+------+------+------+-------

In [ ]:
# To count the total rows in the dataframe.
print('Number of rows: \t', df.count())

# Number of columns can be counted in this way
print('Number of columns: \t', len(df.columns))

# To view the names of the columns
print('\nName of columns: \n', df.columns)

Number of rows: 	 564058
Number of columns: 	 76

Name of columns: 
 ['NU_INSCRICAO', 'NU_ANO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TP_LINGUA', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q0

In [ ]:
# In pandas head() would do similar thing. It is slighly different here. 
# This method is not recommended for the large dataset. 

df.head(5)

[Row(NU_INSCRICAO=200006271946, NU_ANO=2020, TP_FAIXA_ETARIA=11, TP_SEXO='F', TP_ESTADO_CIVIL=1, TP_COR_RACA=2, TP_NACIONALIDADE=1, TP_ST_CONCLUSAO=1, TP_ANO_CONCLUIU=11, TP_ESCOLA=1, TP_ENSINO=None, IN_TREINEIRO=0, CO_MUNICIPIO_ESC=None, NO_MUNICIPIO_ESC=None, CO_UF_ESC=None, SG_UF_ESC=None, TP_DEPENDENCIA_ADM_ESC=None, TP_LOCALIZACAO_ESC=None, TP_SIT_FUNC_ESC=None, CO_MUNICIPIO_PROVA=1501402, NO_MUNICIPIO_PROVA='Bel�m', CO_UF_PROVA=15, SG_UF_PROVA='PA', TP_PRESENCA_CN=0, TP_PRESENCA_CH=0, TP_PRESENCA_LC=0, TP_PRESENCA_MT=0, CO_PROVA_CN=None, CO_PROVA_CH=None, CO_PROVA_LC=None, CO_PROVA_MT=None, NU_NOTA_CN=None, NU_NOTA_CH=None, NU_NOTA_LC=None, NU_NOTA_MT=None, TX_RESPOSTAS_CN=None, TX_RESPOSTAS_CH=None, TX_RESPOSTAS_LC=None, TX_RESPOSTAS_MT=None, TP_LINGUA=1, TX_GABARITO_CN=None, TX_GABARITO_CH=None, TX_GABARITO_LC=None, TX_GABARITO_MT=None, TP_STATUS_REDACAO=None, NU_NOTA_COMP1=None, NU_NOTA_COMP2=None, NU_NOTA_COMP3=None, NU_NOTA_COMP4=None, NU_NOTA_COMP5=None, NU_NOTA_REDACAO=Non

In [ ]:
# There is also take doing the similar thing 

df.take(3)

[Row(NU_INSCRICAO=200006271946, NU_ANO=2020, TP_FAIXA_ETARIA=11, TP_SEXO='F', TP_ESTADO_CIVIL=1, TP_COR_RACA=2, TP_NACIONALIDADE=1, TP_ST_CONCLUSAO=1, TP_ANO_CONCLUIU=11, TP_ESCOLA=1, TP_ENSINO=None, IN_TREINEIRO=0, CO_MUNICIPIO_ESC=None, NO_MUNICIPIO_ESC=None, CO_UF_ESC=None, SG_UF_ESC=None, TP_DEPENDENCIA_ADM_ESC=None, TP_LOCALIZACAO_ESC=None, TP_SIT_FUNC_ESC=None, CO_MUNICIPIO_PROVA=1501402, NO_MUNICIPIO_PROVA='Bel�m', CO_UF_PROVA=15, SG_UF_PROVA='PA', TP_PRESENCA_CN=0, TP_PRESENCA_CH=0, TP_PRESENCA_LC=0, TP_PRESENCA_MT=0, CO_PROVA_CN=None, CO_PROVA_CH=None, CO_PROVA_LC=None, CO_PROVA_MT=None, NU_NOTA_CN=None, NU_NOTA_CH=None, NU_NOTA_LC=None, NU_NOTA_MT=None, TX_RESPOSTAS_CN=None, TX_RESPOSTAS_CH=None, TX_RESPOSTAS_LC=None, TX_RESPOSTAS_MT=None, TP_LINGUA=1, TX_GABARITO_CN=None, TX_GABARITO_CH=None, TX_GABARITO_LC=None, TX_GABARITO_MT=None, TP_STATUS_REDACAO=None, NU_NOTA_COMP1=None, NU_NOTA_COMP2=None, NU_NOTA_COMP3=None, NU_NOTA_COMP4=None, NU_NOTA_COMP5=None, NU_NOTA_REDACAO=Non

In [ ]:
# Selecting a column from the dataframe 

df.select('TP_COR_RACA').show(5)

+-----------+
|TP_COR_RACA|
+-----------+
|          2|
|          3|
|          3|
|          3|
|          3|
+-----------+
only showing top 5 rows



In [ ]:
# Selecting two columns from the dataframe 

df.select('TP_FAIXA_ETARIA', 'TP_ESTADO_CIVIL').show(5)

+---------------+---------------+
|TP_FAIXA_ETARIA|TP_ESTADO_CIVIL|
+---------------+---------------+
|             11|              1|
|             11|              2|
|              4|              2|
|              2|              1|
|              4|              1|
+---------------+---------------+
only showing top 5 rows



In [ ]:
# Droping unwanted row from datafrmae

df.drop('TX_RESPOSTAS_CN').show(5)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESC